In [10]:
import json

In [11]:
data_gt = json.load(open('results_bbox_w_COCO.json')) ## GTp
data_gt = data_gt['annolist']

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import skimage.io
import cv2

In [13]:
edges = [[0,1], [1,2], [2,6], [7,12], [12,11], [11,10], [5,4], [4,3], [3,6], [7,13], [13,14], [14,15], [6,7], [7,8], [8,9]]

In [14]:
def draw_bbox(imageDir, image_name, 
                    rgbPoint_gt = (0, 0, 255), radius = 5,
                    view_gt_points = True, points_id = False, view_bbox = True):
    imagePath = imageDir + '/' + image_name
    
    image = skimage.io.imread(imagePath)
    gt = {}
    size_y = len(image)
    size_x = len(image[0])
    for img in data_gt:
        if img['image'][0]['name'] == image_name:
            gt = img
    if gt == {}:
        assert("no gt")
    #print(gt)
  
    for q in range(len(gt['annorect'])):
        if view_gt_points:
            gt_points = gt['annorect'][q]['annopoints'][0]['point'].copy()
            # pr_points



            # gt_points
            #print(gt_points)
            if view_gt_points:
                our_points = []
                for point in gt_points:
                    our_points.append(point['id'][0])
                gt_points_cpy = gt_points.copy()
                for i in range(0, 16):
                    est = False
                    for p in gt_points:
                        if p['id'][0] == i:
                            est = True
                    if not est:
                        gt_points.append({'id' : [i]})
                gt_points.sort(key = lambda x : x['id'])
                for j, e in enumerate(edges):
                    
                    if e[0] in our_points and e[1] in our_points \
                        and gt_points[e[0]]['x'][0] > 0 and gt_points[e[0]]['y'][0] > 0 \
                        and gt_points[e[1]]['x'][0] > 0 and gt_points[e[1]]['y'][0] > 0:
                        jX = int(gt_points[e[0]]['x'][0])
                        jY = int(gt_points[e[0]]['y'][0])
                        eX = int(gt_points[e[1]]['x'][0])
                        eY = int(gt_points[e[1]]['y'][0])
                        cv2.line(image, (jX, jY),
                              (eX, eY), rgbPoint_gt, 2,
                              lineType=cv2.LINE_AA)
                for point in gt_points_cpy:
                    x = int(point['x'][0])
                    y = int(point['y'][0])
                    cv2.circle(image, (x, y), radius, rgbPoint_gt, -1)
                    if points_id:
                        cv2.putText(image, str(point['id'][0]), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

        if view_bbox:
            bbox_points = gt['annorect'][q]['bboxes']
            xx = int(bbox_points[0])
            yy = int(bbox_points[1])
            w = int(bbox_points[2])
            h = int(bbox_points[3])
            
            cv2.circle(image, (xx, yy), 3, rgbPoint_gt, -1)
            cv2.circle(image, (xx+w, yy), 3, rgbPoint_gt, -1)
            cv2.circle(image, (xx, yy+h), 3, rgbPoint_gt, -1)
            cv2.circle(image, (xx+w, yy+h), 3, rgbPoint_gt, -1)
            cv2.line(image, (xx, yy), (xx+w, yy), rgbPoint_gt, 2, lineType=cv2.LINE_AA)
            cv2.line(image, (xx, yy), (xx, yy+h), rgbPoint_gt, 2, lineType=cv2.LINE_AA)
            cv2.line(image, (xx+w, yy), (xx+w, yy+h), rgbPoint_gt, 2, lineType=cv2.LINE_AA)
            cv2.line(image, (xx, yy+h), (xx+w, yy+h), rgbPoint_gt, 2, lineType=cv2.LINE_AA)
    return image

In [15]:
def showImage(image):
    fig, ax = plt.subplots()

    ax.imshow(image)
    ax.set_title(type(image))

    fig.set_figwidth(60)    #  ширина и
    fig.set_figheight(60)    #  высота "Figure"

    plt.show()

In [16]:
def getListOfImages(data):
    lst = []
    for img in data:
        lst.append(img['image'][0]['name'])
    
    return lst

In [17]:
def main(imgIndex):
    images = getListOfImages(data_gt)
    image= draw_bbox('/home/olga/CenterNet/data/coco', images[imgIndex])
    # view_gt_points = True, points_id = False, view_bbox = True
    
    showImage(image)

In [ ]:
main(1)